# Comparing Classification Models

We want to compare various classification models for the customer dataset. We'll look at logistic regression, LDA, QDA, and $k$-nearest neighbors.

In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# We'll look at logistic regression, LDA, QDA, and KNN
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

# We'll standardize and split data into training/testing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split


# Need to measure "goodness"
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, recall_score, average_precision_score, precision_score
from sklearn.metrics import f1_score, fbeta_score
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay

In [ ]:
# Read in the data and print out its shape
cust = pd.read_csv('./data/customers_clean.csv')
print(cust.shape)

In [ ]:
# Let's drop the following columns:
# cust_id, join_date, last_purchase_date
new_cust = cust.drop(columns=['cust_id','join_date','last_purchase_date'])
new_cust.info()

In [ ]:
# Create dummies and save in new DataFrame
data = pd.get_dummies(new_cust, dtype=int, drop_first=True)
data.info()

## Create "big spender"

We want to convert the $y$ variable, `spend`, into a binary variable where 1 represents a big spender and a 0 otherwise. We can set the cutoff anywhere we want. Looking at the summary statistics from above, let's use \$4,700 (what you told me to do) as the cutoff. We can use the function `pd.cut()`.

In [ ]:
data['big_spender'] = pd.cut(data.spend, bins=[0,4700,10000], right=True, labels=[0,1]).astype(int)
data.info()

In [ ]:
# See summary statistics
data.describe()

## Split Data into Training and Test Sets

Need to first define `X` and `y`. Then we can try train/test split.

In [ ]:
# define the output variable, y
y = data.big_spender

# define the X
# Notice this time we are only dropping the original 'spend'
# and the 'big_spender' columns.
X = data.drop(columns=['spend', 'big_spender'])

In [ ]:
# Time to split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=163)

In [ ]:
# What is the percentage of big spenders in both training and test?
print(f'Training percentage of big spenders is {y_train.mean():.2%}')
print(f'Testing percentage of big spenders is  {y_test.mean():.2%}')

## Scale the Data

Fit only on the training set. Use that fit to transform both the training and test sets.

In [ ]:
# Fit the scaler on just the training X variables
# Let's start with StandardScaler which will center
# each variable at 0 and give each a unit variance (=1)
s_scaler = StandardScaler().fit(X_train)
s_scaler

In [ ]:
# Transform X_train and put in DataFrame
X_train_ss = pd.DataFrame(s_scaler.transform(X_train), columns=X_train.columns)

# Take a look at the DataFrame
X_train_ss.describe()

In [ ]:
# Transform X_test and put in DataFrame
X_test_ss = pd.DataFrame(s_scaler.transform(X_test), columns=X_test.columns)
X_test_ss.describe()

# Function to Look at Different Models

I have write a custom function that will take in different models (classifiers) and compute the different metrics so we can compare the different classifiers easily.

In [ ]:
# define our function
def modelMetrics(classifier, name, X_test, y_test):
    """
    We want to see how the different models react to the same 
    dataset. We should capture multiple metrics for each model.
    
    classifier: the classifier we are capturing metrics for
    
    name: give it a descriptive name
    
    X_test: the X array for the test set
    
    y_test: the output variable (actual) for test set
    """
    retVal = {}
    
    metrics = {}
    predictions = classifier.predict(X_test)
    metrics['a_score'] = accuracy_score(y_test, predictions)
    metrics['r_score'] = recall_score(y_test, predictions)
    metrics['p_score'] = precision_score(y_test, predictions)
    metrics['f1_score'] = f1_score(y_test, predictions)
    metrics['f2_score'] = fbeta_score(y_test, predictions, beta=2)
    metrics['f0.5_score'] = fbeta_score(y_test, predictions, beta=0.5)
    
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    totPositives = y_test.sum()
    totNegatives = len(y_test) - totPositives
    
    # Error rate negatives = false positives / total negatives
    metrics['errorNegatives'] = fp/totNegatives
    # Accuracy for negatives = true negatives / total negatives
    metrics['accNegatives'] = tn/totNegatives
    # Error rate for positives = false negatives / total positives
    metrics['errorPositives'] = fn/totPositives
    # Accuracy for positives = true positives / total positives
    metrics['accPositives'] = tp/totPositives

    metrics['roc_auc_score'] = roc_auc_score(y_test,
                                             classifier.predict_proba(X_test)[:,1])
    metrics['avg_p_score'] = average_precision_score(y_test,
                                                     classifier.predict_proba(X_test)[:,1])
    
    retVal[name] = metrics
    
    return pd.DataFrame(retVal)

# Create a Logistic Model

We will use the scaled data and include all the features.

In [ ]:
# Create a LogisticRegression
logReg = LogisticRegression()

# fit the logistic regression model
logReg.fit(X_train_ss, y_train)

In [ ]:
# Print out the estimated intercept and coefficients
print(logReg.intercept_)
print(logReg.coef_)

In [ ]:
# Get the model metrics using user-defined function
lr_metrics = modelMetrics(logReg, 'logistic regression', X_test_ss, y_test)
lr_metrics

In [ ]:
# Let's make a plot of confusion matrix
ConfusionMatrixDisplay.from_estimator(logReg, X_test_ss, y_test, cmap='cividis')

In [ ]:
# Create an ROC Curve display
RocCurveDisplay.from_estimator(logReg, X_test_ss, y_test)

In [ ]:
# Make a Precision Recall display
PrecisionRecallDisplay.from_estimator(logReg, X_test_ss, y_test)

# Linear Discriminant Analysis
Let's now try LDA. We already imported the appropriate packages for LDA.

In [ ]:
# Create an LDA instance
lda = LinearDiscriminantAnalysis()

In [ ]:
# Fit the LDA model with X and y training set created previously
lda.fit(X_train_ss, y_train)

In [ ]:
# Get the model metrics using user-defined function
lda_metrics = modelMetrics(lda, 'LDA', X_test_ss, y_test)
lda_metrics

# Quadratic Discriminant Analysis
We can also try QDA (already imported package)

In [ ]:
# Create a QDA instance
qda = QuadraticDiscriminantAnalysis()

In [ ]:
# Fit the qda with X and y
qda.fit(X_train_ss, y_train)

In [ ]:
# Get the model metrics using user-defined function
qda_metrics = modelMetrics(qda, 'QDA', X_test_ss, y_test)
qda_metrics

# KNN
Might as well try $k$-nearest neighbors. You pick $k$, so it will be your fault not mine.

In [ ]:
# Let's use k=??
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# fit the model
knn.fit(X_train_ss, y_train)

In [ ]:
# Get the model metrics using user-defined function
knn3_metrics = modelMetrics(knn, 'KNN-3', X_test_ss, y_test)
knn3_metrics

In [ ]:
# Let's try another value for k ... k=??
knn20 = KNeighborsClassifier(n_neighbors=20)
knn20.fit(X_train_ss, y_train)

In [ ]:
# Get the model metrics using user-defined function
knn20_metrics = modelMetrics(knn20, 'KNN-20', X_test_ss, y_test)
knn20_metrics

In [ ]:
# Put all metrics in one DataFrame to examine
all_dfs = [lr_metrics, lda_metrics, qda_metrics, knn3_metrics, knn20_metrics]

In [ ]:
# Look at all metrics together
all_metrics = pd.concat(all_dfs, axis=1)
all_metrics

In [ ]:
print(all_metrics.to_markdown())

## Find the "Best" $k$ for KNN

We can systematically search various values of $k$ to find the one that gives us the best performance (will need to define) using `GridSearchCV`. What `GridSearchCV` does is train the model multiple times on a range of parameters we specify. In our case that would be $k$.

In [ ]:
# import GridSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create a new knn model
knn_base = KNeighborsClassifier()

In [ ]:
# Define our search grid
# This is a dictionary of values for k we want search thru
param_grid = {'n_neighbors': np.arange(1, 25)}

In [ ]:
# Let's run the grid search
knnCV = GridSearchCV(knn_base, param_grid, cv=5, scoring='recall')

In [ ]:
knnCV.fit(X_train_ss, y_train)

In [ ]:
knnCV.best_score_

In [ ]:
knnCV.best_params_

In [ ]:
knnCV.cv_results_

In [ ]:
# Let's run the grid search
knnCV = GridSearchCV(knn_base, param_grid, cv=5, scoring='average_precision')

In [ ]:
knnCV.fit(X_train_ss, y_train)

In [ ]:
knnCV.best_params_

In [ ]:
knnCV.best_score_

## Threshold-Moving

So far, we've been just keeping the default cut-off probability, which happens to be 0.5, for determining whether our binary class should be a 0 (non-big spender) or a 1 (big spender). We can change the **threshold** for what is considered class 0 or 1. This is called **threshold-moving**.

Let's change the threshold from the default of 0.5 to 0.2 for the LDA model and see what happens.

In [ ]:
# Let's create a function that will do all the hard work for us (hopefully)
def compare_metrics(classifier, X_test, y_test):
    results = {}
#     retVal = []
    # Loop over different thresholds
    for threshold in np.arange(0.1, 0.91, 0.01):
        yProb = classifier.predict_proba(X_test)
        tn, fp, fn, tp = confusion_matrix(y_test, yProb[:,1]>threshold).ravel()
        totPositives = y_test.sum()
        totNegatives = len(y_test) - totPositives

        # Accuracy for negatives = true negatives / total negatives
        accNegatives = tn/totNegatives
        # Accuracy for positives = true positives / total positives
        accPositives = tp/totPositives
        
        results[threshold] = [accPositives, accNegatives]
#         # Add result to retVal list
#         retVal.append((threshold, accPositives, accNegatives))
        retVal = pd.DataFrame.from_dict(results, orient='index', columns=['Accuracy Big Spenders','Accuracy Others'])
        
    return retVal

In [ ]:
# Let's try it with logistic regression
values = compare_metrics(logReg, X_test_ss, y_test)
values

In [ ]:
print(values.to_string())

In [ ]:
# The scope of these changes made to
# pandas settings are local to with statement.
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(values)

In [ ]:
print(values.to_markdown())

In [ ]:
sns.relplot(x=values.index, y='Accuracy Big Spenders', data=values)
sns.relplot(x=values.index, y='Accuracy Others', data=values)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(values.index, y=values['Accuracy Big Spenders'], label='big spender accuracy')
ax.scatter(values.index, values['Accuracy Others'], label='other accuracy', marker='s', color='orange')
ax.legend()
ax.set_xlabel('Cut-off value')
ax.set_ylabel('Accuracy')